In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors
import sklearn
import re
import tqdm


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Atharva
[nltk_data]     J\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Atharva
[nltk_data]     J\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Atharva
[nltk_data]     J\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Atharva
[nltk_data]     J\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Import data

In [ ]:
review_data = pd.read_csv("D:\\6461_AdvancedAIforBusinessApplications\\Practice\\Datasets\\imdb_movie_reviews_50k.csv")

In [ ]:
review_data['sentiment'] = pd.get_dummies(review_data['sentiment'])['positive']

In [ ]:
review_data.head()

In [ ]:
review_data.shape

### Preprocess raw text data

In [ ]:
## stopword removal, stemming/lemmatization
stopwords = nltk.corpus.stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()
    
def preprocess(review):
    review = re.sub('[^a-zA-Z0-9]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    return review

In [ ]:
## preprocessing
review_data['review'] = review_data['review'].apply(preprocess)

In [ ]:
review_data

In [ ]:
review_data.to_csv("D:\\6461_AdvancedAIforBusinessApplications\\Practice\\Datasets\\review_data_preprocessed_and_lemmatized.csv", index=False)

In [22]:
# review_data = pd.read_csv("D:\\6461_AdvancedAIforBusinessApplications\\Practice\\Datasets\\review_data_preprocessed_and_lemmatized.csv")

In [3]:
def prep_for_word2vec(review):
    return gensim.utils.simple_preprocess(review)

### Initialize Word2Vec

In [4]:
# wv = api.load("word2vec-google-news-300")
wv = KeyedVectors.load_word2vec_format('D:\\6461_AdvancedAIforBusinessApplications\\Practice\\Datasets\\GoogleNews-vectors-negative300.bin', binary=True)

In [23]:
review_data['review'] = review_data['review'].apply(prep_for_word2vec)

In [ ]:
# Train Word2Vec Model

In [24]:
wv_model = gensim.models.Word2Vec(review_data['review'], window=5, min_count=2)

In [7]:
wv_model.wv

In [25]:
def avg_word2vec(review):
    return list(np.mean([wv_model.wv[word] for word in review if word in wv_model.wv.index_to_key], axis=0))

In [26]:
review_data['review'] = review_data['review'].apply(avg_word2vec)

In [27]:
review_data.head()

,review,sentiment
0,"[-0.026128303, 0.11902674, -0.15354283, -0.212...",1
1,"[0.08929897, -0.48858237, -0.11523942, -0.1937...",1
2,"[0.23162502, -0.3230856, -0.36416396, -0.35160...",1
3,"[0.11159298, 0.03788822, -0.34463963, -0.35367...",0
4,"[0.1598897, 0.04216536, 0.024150012, -0.193933...",1


In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [29]:
rfc = RandomForestClassifier()

In [30]:
X_train, X_test, y_train, y_test = train_test_split(list(review_data['review']), review_data['sentiment'], random_state=42)

In [31]:
X_train

[[0.1644284,
  -0.31520733,
  -0.25598267,
  -0.14904654,
  -0.1974724,
  -0.5074252,
  0.29117185,
  0.40064475,
  0.22815521,
  -0.30270556,
  0.029775742,
  0.30218393,
  -0.40123323,
  0.3668935,
  0.17097685,
  0.31079134,
  0.25017434,
  0.11555557,
  -0.45854145,
  -0.21405204,
  0.098532826,
  0.43139485,
  0.5604133,
  -0.14394712,
  0.19436958,
  0.10903026,
  -0.6109384,
  0.42713666,
  0.047534954,
  0.24444671,
  0.025660748,
  -0.038607065,
  0.37859243,
  -0.22456133,
  0.03218978,
  -0.08355109,
  0.071424566,
  -0.06857964,
  0.39999744,
  0.23952115,
  -0.16450414,
  -0.29403692,
  -0.0068574753,
  0.13125515,
  0.19320644,
  -0.026687741,
  -0.25521785,
  0.27960965,
  0.58110857,
  0.25780442,
  -0.14326026,
  0.29984567,
  -0.28470096,
  -0.24968792,
  0.19520023,
  0.019870412,
  0.16968985,
  -0.2590736,
  -0.30649024,
  0.03155484,
  -0.0020189937,
  -0.014008023,
  0.046553504,
  -0.18923359,
  -0.16511552,
  0.5965466,
  -0.11230554,
  0.52205485,
  -0.4535703

In [32]:
rfc_model = rfc.fit(X_train, y_train)

In [33]:
y_pred = rfc_model.predict(X_test)

In [35]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [37]:
accuracy_score(y_test,y_pred)

0.8344

In [38]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83      6027
           1       0.85      0.83      0.84      6473

    accuracy                           0.83     12500
   macro avg       0.83      0.83      0.83     12500
weighted avg       0.83      0.83      0.83     12500

